# Import pacakges

In [1]:
import pyjs

In [2]:

pyjs.js.Function("""
  globalThis["_window"] = this || self || window;
  const url = globalThis.origin + "/roslib.js"
  console.log(url);
  importScripts(url);
""")()


In [3]:
pyjs.js.Function("""
  const url = globalThis.origin + "/pre.js"
  console.log(url);
  importScripts(url);
""")()


In [4]:
import os
os.environ["ROS_DOMAIN_ID"]="0"

import asyncio
import pyjs
import rclpy

from rclpy.utilities import get_default_context
from rclpy.signals import SignalHandlerOptions
from rclpy.signals import install_signal_handlers
from rclpy.node import Node
from std_msgs.msg import String

# Create a Subscriber class

In [5]:
class MinimalSubscriber(Node):
  def __init__(self, name, topics):
    super().__init__(name)
    self._subs = [self.create_subscription(
      String,
      topic,
      self.listener_callback,
      10) for topic in topics]
    self._subs  # prevent unused variable warning
    self._name = name

  def listener_callback(self, msg):
    print(self._name + " heard: \"" + msg.data + "\"")


# Async spin functions

In [6]:
running = True

async def spin_subscriber(sub):
  while running:
    rclpy.spin_once(sub, timeout_sec=0)
    await asyncio.sleep(0.01)

async def publish_messages(pub, text):
  count = 0
  while running:
    msg = String()
    msg.data = text + str(count)
    count = count + 1
    pub.publish(msg)
    await asyncio.sleep(1)


# Create ROS context

In [7]:
context = get_default_context()
context.init(args=None, domain_id=None)
context.ok()

True

# Create publishers and subscribers

In [8]:
node = rclpy.create_node('minimal_publisher', context=context, use_global_arguments=False, start_parameter_services=True)

pub_a = node.create_publisher(String, 'topic_a', 10)
pub_b = node.create_publisher(String, 'topic_b', 10)
sub_1 = MinimalSubscriber('minimal_subscriber_test', ['topic_a'])
sub_2 = MinimalSubscriber('minimal_subscriber_test_b', ['topic_b'])

# Start publisher and subscriber tasks

In [10]:
running = True
async def stopper():
    print("Running nodes for 60 seconds")
    global running
    await asyncio.sleep(60)
    print("Stopping all tasks")
    running = False
    
asyncio.create_task(stopper())
asyncio.create_task(publish_messages(pub_a, 'Hello From A '))
asyncio.create_task(publish_messages(pub_b, 'Hello From B '))
asyncio.create_task(spin_subscriber(sub_1))
asyncio.create_task(spin_subscriber(sub_2))


<Task pending name='Task-6' coro=<spin_subscriber() running at /tmp/xpython_42/2174627779.py:3>>

Running nodes for 60 seconds
Stopping all tasks
Stopping all tasks
